In [1]:
import numpy as np

import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.max_rows = 400

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid")

from matplotlib import rcParams

rcParams['figure.figsize'] = 10, 8
rcParams['font.size'] = 16
rcParams['axes.labelsize'] = 14
rcParams['xtick.labelsize'] = 13
rcParams['ytick.labelsize'] = 13
rcParams['legend.fontsize'] = 15

---

# Recruit Restaurant Visitor Forecasting

Проходим по ссылке, скачиваем данные:

https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting

Ребята просят предсказать, сколько человек к ним заявится в следующие недели, в каждый из дней.

Положите все ваши данные в папку _"data"_ рядом с этой тетрадкой. Дальше просто прогоняйте эту тетрадку и смотрите на то, что я делаю с данными: какую табличку соединяю и почему.

Готовы? Погнали! Все по-серьёзке, не по синьке : )

In [2]:
!ls data/

air_reserve.csv       date_info.csv         sample_submission.csv
air_store_info.csv    hpg_reserve.csv       store_id_relation.csv
air_visit_data.csv    hpg_store_info.csv


# Таблички

Что имеем на руках? Информацию о заведениях из двух различных источников:

* "Hot Pepper Gourmet" - сервис по оценке и ревью ресторанов;
* "AirREGI" - сервис облуживания точек продаж ресторанов (ну вы поняли, всякие оплаты через карты проходях через них, - оттуда и данные).

Из-за того, что источника два, информация немного разнится и доставит нам проблем при слиянии таблиц в одну, но что поделать: _c'est la vie_ (_вдали играет грустный аккордеон..._).

Данные временные: имеем историю с __1.01.2016__ по __01.04.2017__ года, и в отложенную выборку попадают следующие пара недель апреля-мая.

Авторы также предупреждают, что в отложенную выборку попадает ["Золотая неделя"](https://ru.wikipedia.org/wiki/%D0%97%D0%BE%D0%BB%D0%BE%D1%82%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B4%D0%B5%D0%BB%D1%8F), так что это наверняка скажется на характере посещений ресторанов.

Давайте посмотрим повнимательнее на таблички и начнем их сишвать в одну матрицу "объекты-признаки". В этот раз я помогу прям до конца, но это в последний раз!

## Air Visit Data и ...sample submission?

__air_visit_data.csv__

This file contains historical visit data for the air restaurants.

* `air_store_id`
* `visit_date` - the date
* `visitors` - the number of visitors to the restaurant on the date

In [3]:
# по сути sample_submission - и есть test set, если его причесать...
test = pd.read_csv('data/sample_submission.csv')
 
# ... и привести к виду air_visit_data, который, по сути, train.
train = pd.read_csv('data/air_visit_data.csv')

In [4]:
test['air_store_id'] = test.id.str.split('_').str.get(0) + '_' + test.id.str.split('_').str.get(1)
test['visit_date'] = test.id.str.split('_').str.get(2)
test['vis'] = test['visitors']
test.drop(['id', 'visitors'],axis=1, inplace=True)
test['visitors'] = test['vis']
test.drop(['vis'],axis=1, inplace=True)

---

In [5]:
train.head(1)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25


In [6]:
test.head(1)

,air_store_id,visit_date,visitors
0,air_00a91d42b08b08d9,2017-04-23,0


Отлично, понятно, теперь у нас формат в train и test-выборках тот же, надо лишь предсказать __visitors__ - нашу целевую переменную.

---

## Air Store Info

__air_store_info.csv__

This file contains information about select air restaurants. Column names and contents are self-explanatory.

* `air_store_id`
* `air_genre_name`
* `air_area_name`
* `latitude`
* `longitude`

_Note:_ latitude and longitude are the latitude and longitude of the area to which the store belongs

In [6]:
air_store_info = pd.read_csv('data/air_store_info.csv')

In [12]:
air_store_info.head(1)

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852


Это по сути новые фичи, берем и пристыковываем:

In [13]:
train = pd.merge(train, air_store_info) 
test  = pd.merge(test,  air_store_info) 

### ID

Сделаем __`id`__ в виде склеивания __`air_store_id`__ и дня посещения заведения __`visit_date`__. 

Он пригодится нам впоследствии для слияния таблиц: каждое его уникальное значение - как составной ключ в SQL - __отражает данные в данном ресторане в определенный день года__.

In [14]:
train['id'] = train['air_store_id'] + '_' + train['visit_date']
test['id']  = test['air_store_id']  + '_' + test['visit_date']

---

## Air Reserve

__air_reserve.csv__

This file contains reservations made in the air system. Note that the reserve_datetime indicates the time when the reservation was created, whereas the visit_datetime is the time in the future where the visit will occur.

* `air_store_id` - the restaurant's id in the air system
* `visit_datetime` - the time of the reservation
* `reserve_datetime` - the time the reservation was made
* `reserve_visitors` - the number of visitors for that reservation

In [15]:
air_reserve = pd.read_csv('data/air_reserve.csv')

In [16]:
air_reserve.head(1)

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1


### ID

Вот отсюда как раз __`visit_date`__ и может быть выделен. А заодно - и __`id`__ по логике, описанной выше, чтобы merge состоялся :)

In [17]:
air_reserve['visit_date'] =  air_reserve.visit_datetime.str.split(' ').str.get(0)
air_reserve['id'] = air_reserve['air_store_id'] + '_' + air_reserve['visit_date']

### Feature Engineering

In [18]:
# to datetime conversion
air_reserve['visit_datetime']   = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])

* За сколько часов был заказан стол?

In [19]:
air_reserve['period'] = air_reserve['visit_datetime'] - air_reserve['reserve_datetime']
air_reserve['period'] = air_reserve['period'] / np.timedelta64(1, 'h')

* В день пятницы? Или выходного дня?

In [20]:
air_reserve['is_visit_on_friday'] = (air_reserve.visit_datetime.dt.dayofweek == 4).astype(np.int64)
air_reserve['is_visit_on_weeknd'] = (air_reserve.visit_datetime.dt.dayofweek > 4).astype(np.int64)

<div class="panel panel-info">
    <div class="panel-heading">
        <h4 class="panel-title">Как насчет подумать?</h4> 
    </div>
    Здесь ты, очевидно, можешь добавить своих новых признаков, если что-нибудь придумаешь:
</div>

In [ ]:
# Ваш код здесь

In [22]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date,id,period,is_visit_on_friday,is_visit_on_weeknd
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01,air_877f79706adbfb06_2016-01-01,3.0,1,0
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01,air_db4b38ebe7a7ceff_2016-01-01,0.0,1,0
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01,air_db4b38ebe7a7ceff_2016-01-01,0.0,1,0
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01,air_877f79706adbfb06_2016-01-01,4.0,1,0
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01,air_db80363d35f10926_2016-01-01,19.0,1,0


### Аггрегация по дням

In [23]:
air_reserve_agg = air_reserve.pivot_table(index=['id'], 
                                          values=['reserve_visitors', 
                                                  'period',
                                                  'is_visit_on_friday',
                                                  'is_visit_on_weeknd',
                                                 ],
                                          aggfunc={
                                              'reserve_visitors': 'sum', 
                                              'period': 'mean',
                                              'is_visit_on_friday': 'mean',
                                              'is_visit_on_weeknd': 'mean',
                                          }, fill_value=0)

air_reserve_agg.reset_index(inplace=True)

In [24]:
air_reserve_agg.head()

,id,is_visit_on_friday,is_visit_on_weeknd,period,reserve_visitors
0,air_00a91d42b08b08d9_2016-10-31,0,0,4.0,2
1,air_00a91d42b08b08d9_2016-12-05,0,0,100.0,9
2,air_00a91d42b08b08d9_2016-12-14,0,0,153.0,18
3,air_00a91d42b08b08d9_2016-12-17,0,1,147.0,2
4,air_00a91d42b08b08d9_2016-12-20,0,0,51.0,4


In [25]:
train = train.merge(air_reserve_agg, how='left')
test  = test.merge(air_reserve_agg,  how='left')

---

## Store ID Relation

__store_id_relation.csv__

This file allows you to join select restaurants that have both the air and hpg system.

* `hpg_store_id`
* `air_store_id`

Вот тут надо быть внимательным, начинается геморрой: об одном и том же заведении может быть информация и у air, и у hpg. 

Авторы красавчики, и максимально совместики формат. Однако могут встречаться заведения, которые есть только у одних и которых нет у других.

In [27]:
store_id_relation = pd.read_csv('data/store_id_relation.csv')

---

## HPG Reserve

__hpg_reserve.csv__

This file contains reservations made in the hpg system.

* `hpg_store_id` - the restaurant's id in the hpg system
* `visit_datetime` - the time of the reservation
* `reserve_datetime` - the time the reservation was made
* `reserve_visitors` - the number of visitors for that reservation

In [28]:
hpg_reserve = pd.read_csv('data/hpg_reserve.csv')

In [29]:
hpg_reserve.head(1)

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1


__NB__: Табличка, казалось бы, такая же, как и Air Reserve. Правильно. Но если мы будем проводить те же операции, то при слиянии возникнут проблемы, потому что, например, __`reserve_visitors`__ у нас уже есть. Поэтому будем добавлять префикс __hpg__.

In [30]:
hpg_reserve.rename(columns={'reserve_visitors': 'hpg_reserve_visitors'}, inplace=True)

---

Весь __`hpg_reserve`__ нам не нужен: только те, у кого есть пара с air-id:

In [31]:
hpg_air_reserve = hpg_reserve.merge(store_id_relation, how='left').dropna()

### ID

In [32]:
hpg_air_reserve['visit_date'] = hpg_air_reserve['visit_datetime'].str.split(' ').str.get(0)
hpg_air_reserve['id'] = hpg_air_reserve['air_store_id'] + '_' + hpg_air_reserve['visit_date']

### Feature engineering

In [33]:
# to datetime conversion
hpg_air_reserve['visit_datetime']   = pd.to_datetime(hpg_air_reserve['visit_datetime'])
hpg_air_reserve['reserve_datetime'] = pd.to_datetime(hpg_air_reserve['reserve_datetime'])

* За сколько часов был заказан стол?

In [34]:
hpg_air_reserve['hpg_period'] = hpg_air_reserve['visit_datetime'] - hpg_air_reserve['reserve_datetime']
hpg_air_reserve['hpg_period'] = hpg_air_reserve['hpg_period'] / np.timedelta64(1, 'h')

* В день пятницы? Или выходного дня?

In [35]:
hpg_air_reserve['hpg_is_visit_on_friday'] = (hpg_air_reserve.visit_datetime.dt.dayofweek == 4).astype(np.int64)
hpg_air_reserve['hpg_is_visit_on_weeknd'] = (hpg_air_reserve.visit_datetime.dt.dayofweek > 4).astype(np.int64)

### Аггрегация по дням

In [36]:
hpg_air_reserve_agg = hpg_air_reserve.pivot_table(index=['id'], 
                                                  values=[
                                                      'hpg_reserve_visitors',  
                                                      'hpg_period',
                                                      'hpg_is_visit_on_friday',
                                                      'hpg_is_visit_on_weeknd',
                                                  ],
                                                  aggfunc={
                                                      'hpg_reserve_visitors': 'sum', 
                                                      'hpg_period': 'mean',
                                                      'hpg_is_visit_on_friday': 'mean',
                                                      'hpg_is_visit_on_weeknd': 'mean',
                                                  }, fill_value=0)

hpg_air_reserve_agg.reset_index(inplace=True)

In [37]:
hpg_air_reserve_agg.head()

,id,hpg_is_visit_on_friday,hpg_is_visit_on_weeknd,hpg_period,hpg_reserve_visitors
0,air_00a91d42b08b08d9_2016-01-14,0,0,77.0,2
1,air_00a91d42b08b08d9_2016-01-15,1,0,142.0,4
2,air_00a91d42b08b08d9_2016-01-16,0,1,70.0,2
3,air_00a91d42b08b08d9_2016-01-22,1,0,82.0,2
4,air_00a91d42b08b08d9_2016-01-29,1,0,144.0,5


In [38]:
train = train.merge(hpg_air_reserve_agg, how='left')
test  = test.merge(hpg_air_reserve_agg,  how='left')

---

## Date Info

__date_info.csv__

This file gives basic information about the calendar dates in the dataset.

* `calendar_date`
* `day_of_week`
* `holiday_flg` - is the day a holiday in Japan

In [39]:
date_info = pd.read_csv('data/date_info.csv')
date_info.rename(columns={'calendar_date': 'visit_date'}, inplace=True)

In [40]:
date_info.head()

,visit_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [41]:
train = train.merge(date_info, how='left')
test  = test.merge(date_info,  how='left')

---

Итого:

In [42]:
train.shape, test.shape

((252108, 18), (32019, 18))

# Feature Engineering

<div class="panel panel-info" style="margin: 20px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задание 1. Новые признаки</h3> 
    </div>
</div>

Добавляй! Помни, чему тебя учил Ольферук!

In [ ]:
# Ваш код здесь

# Визуализация

<div class="panel panel-info" style="margin: 20px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задание 2. Давно не рисовали?</h3> 
    </div>
</div>

Время рисовать! Не недооценивай важность графиков. Хорошая картинка показательнее многих команд и статистических тестов. Думай, исследуй, набивай руку, РАБОТАЙ, СУКА, одним словом.

In [ ]:
# Ваш код здесь

# Кодирование и подготовка выборки

<div class="panel panel-info" style="margin: 20px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задание 3. Че там дальше?</h3> 
    </div>
</div>

Признаки кодировать можно, как ты помнишь, по-разному:

1. `LabelEncoder` - парень, который превращает значения в циферки по порядку.
2. `LabelBinarizer` - one-hot-кодирование, ты помнишь? Единичка где-то в векторе, размером со все уникальные значения признака.
3. `CountVectorizer` - мешок слов. Как one-hot, но единичек может быть много. Возможно, пойдет для кодирования региона в нашем случае. Хз.

И сформируй выборку. __visitors__ предсказываем, - помнишь?

In [ ]:
# Ваш код здесь

# Модель

## Train & Evaluate

<div class="panel panel-info" style="margin: 20px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Задание 4. Я у мамы дата-сайентист</h3> 
    </div>
</div>

Ну ты понял. Самая безмозглая часть работы настала. 

1. Импортируешь свой любимый классификатор.
2. fit.
3. predict.

На трейне можешь измерять качество, как хочешь.

Когда наиграешься, выкинь из `test` столбец __`visitors`__, и предскажи его с помощью модели. Можешь как-то так:

```python
    answers = clf.predict(X_test)
    submission = pd.read_csv('data/sample_submission.csv')
    submission.visitors = answers
    submission['visitors'] = submission['visitors'].apply(lambda x: 0 if x < 0 else x) # ага, не забудь перестраховаться
    submission.to_csv('../output/first_pred.csv', index=False)
```

<div class="panel panel-success" style="margin: 50px 0 0 0">
    <div class="panel-heading">
        <h3 class="panel-title">Финиш?</h3> 
    </div>
</div>

Добрался до финиша? Здорово! Но еще далеко не конец.

Отправляй результаты на kaggle. Наверняка будет не топовый результат. Почему? Стоит вернуться и проанализировать. Стоит обсудить. Стоит посмотреть, что у других. Обязательно делись находками, графиками, новыми мыслями в чат. 

Вместе мы точно осилим!